# TA-Lib Tutorial

"TA-Lib is widely used by trading software developers requiring to perform technical analysis of financial market data."

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# imports
import pandas as pd
import pandas_datareader.data as pdr
import datetime
import talib
from talib.abstract import *
from talib import MA_Type

# format price data
pd.options.display.float_format = '{:0.2f}'.format

Some global data

In [3]:
symbol = 'SPY'
start = datetime.datetime(2018, 1, 1)
end = datetime.datetime.now()

Fetch timeseries

In [4]:
ts = pdr.DataReader(symbol, 'yahoo', start, end)
ts.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-02-22,389.62,386.74,387.06,387.03,67160000.00,387.03
2021-02-23,388.95,380.20,384.66,387.50,106997200.00,387.50
2021-02-24,392.23,385.27,386.33,391.77,72226600.00,391.77
2021-02-25,391.88,380.78,390.41,382.33,146086500.00,382.33
2021-02-26,385.58,378.23,384.35,380.36,152534900.00,380.36


In [5]:
def _adj_column_names(ts):
    """
    ta-lib expects columns to be lower case; to be consistent,
    change date index
    """
    ts.columns = [col.lower().replace(' ','_') for col in ts.columns]
    ts.index.names = ['date']
    return ts

ts = _adj_column_names(ts)

Select timeseries between start and end.

### Get info about TA-Lib

In [6]:
print('There are {} TA-Lib functions!'.format(len(talib.get_functions())))

There are 158 TA-Lib functions!


Here is a complete listing of the functions by group:

In [7]:
for group, funcs in talib.get_function_groups().items():
    print(group)
    print('-----------------------------------------')
    for func in funcs:
        f = Function(func)
        print('{} - {}'.format(func, f.info['display_name']))
    print()

Cycle Indicators
-----------------------------------------
HT_DCPERIOD - Hilbert Transform - Dominant Cycle Period
HT_DCPHASE - Hilbert Transform - Dominant Cycle Phase
HT_PHASOR - Hilbert Transform - Phasor Components
HT_SINE - Hilbert Transform - SineWave
HT_TRENDMODE - Hilbert Transform - Trend vs Cycle Mode

Math Operators
-----------------------------------------
ADD - Vector Arithmetic Add
DIV - Vector Arithmetic Div
MAX - Highest value over a specified period
MAXINDEX - Index of highest value over a specified period
MIN - Lowest value over a specified period
MININDEX - Index of lowest value over a specified period
MINMAX - Lowest and highest values over a specified period
MINMAXINDEX - Indexes of lowest and highest values over a specified period
MULT - Vector Arithmetic Mult
SUB - Vector Arithmetic Substraction
SUM - Summation

Math Transform
-----------------------------------------
ACOS - Vector Trigonometric ACos
ASIN - Vector Trigonometric ASin
ATAN - Vector Trigonometric AT

### Get info about a specific TA-Lib function

There are 2 different API that are available with talib, namely Function API and Abstract API.  For the Function API, you pass in a price series.  For the Abstract API, you pass in a collection of named inputs: 'open', 'high', 'low', 'close', and 'volume'.  One or more of these may be used as defaults, but can be changed with the 'price' parameter.  

Print the function instance to get documentation.  We see that SMA has the parameter 'timeperiod' with default '30'.  The input_arrays can be a dataframe with columns named 'open', 'high', 'low', 'close', and 'volume'.

In [8]:
print(SMA)

SMA([input_arrays], [timeperiod=30])

Simple Moving Average (Overlap Studies)

Inputs:
    price: (any ndarray)
Parameters:
    timeperiod: 30
Outputs:
    real


More information is available through the 'info' property.  We observe here that the default price used is 'close'.  This can be changed by setting 'price' in the function call, e.g. price='open'.

In [9]:
print(SMA.info)

{'name': 'SMA', 'group': 'Overlap Studies', 'display_name': 'Simple Moving Average', 'function_flags': ['Output scale same as input'], 'input_names': OrderedDict([('price', 'close')]), 'parameters': OrderedDict([('timeperiod', 30)]), 'output_flags': OrderedDict([('real', ['Line'])]), 'output_names': ['real']}


If we just want to see the inputs, we can print the input_names property.

In [10]:
print(SMA.input_names)

OrderedDict([('price', 'close')])


If we just want to see the parameters, we can print the paramters property.

In [11]:
print(SMA.parameters)

OrderedDict([('timeperiod', 30)])


If we just want to see the outputs, we can print the output_names property.

In [12]:
print(SMA.output_names)

['real']


### Create a technical indicator using talib

Create technical indicator: SMA (using defaults: timeperiod=30, price='close')

In [13]:
sma = SMA(ts)
sma.tail()

date
2021-02-22   383.76
2021-02-23   383.97
2021-02-24   384.40
2021-02-25   384.52
2021-02-26   384.54
dtype: float64

Create technical indicator: SMA (using: timeperiod=200, price='close')

In [14]:
sma200 = SMA(ts, timeperiod=200)
sma200.tail()

date
2021-02-22   342.20
2021-02-23   342.70
2021-02-24   343.20
2021-02-25   343.65
2021-02-26   344.12
dtype: float64

Create technical indicator: SMA (using: timeperiod=200, price='open')

In [15]:
sma200 = SMA(ts, timeperiod=200, price='open')
sma200.tail()

date
2021-02-22   342.24
2021-02-23   342.73
2021-02-24   343.20
2021-02-25   343.70
2021-02-26   344.16
dtype: float64

### Add a technical indicator to a timeseries

In [16]:
ts['sma200'] = sma200
ts.tail()

,high,low,open,close,volume,adj_close,sma200
date,,,,,,,
2021-02-22,389.62,386.74,387.06,387.03,67160000.00,387.03,342.24
2021-02-23,388.95,380.20,384.66,387.50,106997200.00,387.50,342.73
2021-02-24,392.23,385.27,386.33,391.77,72226600.00,391.77,343.20
2021-02-25,391.88,380.78,390.41,382.33,146086500.00,382.33,343.70
2021-02-26,385.58,378.23,384.35,380.36,152534900.00,380.36,344.16


### Try another one

Commodity Channel Index

In [17]:
print(CCI)

CCI([input_arrays], [timeperiod=14])

Commodity Channel Index (Momentum Indicators)

Inputs:
    prices: ['high', 'low', 'close']
Parameters:
    timeperiod: 14
Outputs:
    real


In [18]:
print(CCI.input_names)

OrderedDict([('prices', ['high', 'low', 'close'])])


In [19]:
print(CCI.parameters)

OrderedDict([('timeperiod', 14)])


In [20]:
cci = CCI(ts)
ts['cci'] = cci
ts.tail()

,high,low,open,close,volume,adj_close,sma200,cci
date,,,,,,,,
2021-02-22,389.62,386.74,387.06,387.03,67160000.00,387.03,342.24,-16.31
2021-02-23,388.95,380.20,384.66,387.50,106997200.00,387.50,342.73,-92.48
2021-02-24,392.23,385.27,386.33,391.77,72226600.00,391.77,343.20,12.88
2021-02-25,391.88,380.78,390.41,382.33,146086500.00,382.33,343.70,-173.20
2021-02-26,385.58,378.23,384.35,380.36,152534900.00,380.36,344.16,-218.21


### Now for something a little more difficult

Bollinger Bands

In [21]:
print(BBANDS)

BBANDS([input_arrays], [timeperiod=5], [nbdevup=2], [nbdevdn=2], [matype=0])

Bollinger Bands (Overlap Studies)

Inputs:
    price: (any ndarray)
Parameters:
    timeperiod: 5
    nbdevup: 2
    nbdevdn: 2
    matype: 0 (Simple Moving Average)
Outputs:
    upperband
    middleband
    lowerband


In [22]:
print(BBANDS.input_names)

OrderedDict([('price', 'close')])


In [23]:
print(BBANDS.parameters)

OrderedDict([('timeperiod', 5), ('nbdevup', 2), ('nbdevdn', 2), ('matype', 0)])


Print the available moving average types

In [24]:
attributes = [attr for attr in dir(MA_Type) 
              if not attr.startswith('__')]
attributes

['DEMA', 'EMA', 'KAMA', 'MAMA', 'SMA', 'T3', 'TEMA', 'TRIMA', 'WMA', '_lookup']

In [25]:
MA_Type.__dict__

{'_lookup': {0: 'Simple Moving Average',
  1: 'Exponential Moving Average',
  2: 'Weighted Moving Average',
  3: 'Double Exponential Moving Average',
  4: 'Triple Exponential Moving Average',
  5: 'Triangular Moving Average',
  6: 'Kaufman Adaptive Moving Average',
  7: 'MESA Adaptive Moving Average',
  8: 'Triple Generalized Double Exponential Moving Average'}}

In [26]:
print(MA_Type[MA_Type.DEMA])

Double Exponential Moving Average


Set timeperiod=20 and matype=MA_Type.EMA

In [27]:
#upper, middle, lower = BBANDS(ts, timeperiod=20, matype=MA_Type.EMA)
#(for some reason, the abstract API doesn't work for BBANDS, so use the function API)

upper, middle, lower = talib.BBANDS(ts.close, timeperiod=20, matype=MA_Type.EMA)
ts['upper'] = upper; ts['middle'] = middle; ts['lower'] = lower
ts.tail()

,high,low,open,close,volume,adj_close,sma200,cci,upper,middle,lower
date,,,,,,,,,,,
2021-02-22,389.62,386.74,387.06,387.03,67160000.00,387.03,342.24,-16.31,399.38,386.44,373.49
2021-02-23,388.95,380.20,384.66,387.50,106997200.00,387.50,342.73,-92.48,399.50,386.54,373.58
2021-02-24,392.23,385.27,386.33,391.77,72226600.00,391.77,343.20,12.88,400.23,387.04,373.84
2021-02-25,391.88,380.78,390.41,382.33,146086500.00,382.33,343.70,-173.20,398.80,386.59,374.38
2021-02-26,385.58,378.23,384.35,380.36,152534900.00,380.36,344.16,-218.21,397.86,386.00,374.13
